In [ ]:
import datetime
import pandas_datareader.data as web
from sarimaxModel import sarimaxPrdict
from sarimaxModel import quickParameters
from sarimaxModel import selectParameters
from sarimaxModel import predictbyticker
import pandas as pd
from sarimaxModel import forcastStocks
from loadstocks import loadStocksByTickers
from loadstocks import loadAuNotNaTickersFromYahooExcel
from sarimaxModel import predictbyticker
import pandas as pd
from sarimaxModel import forcastStocks
from loadstocks import loadStocksByTickers
from loadstocks import loadAuNotNaTickersFromYahooExcel
# We will look at stock prices over the past year, starting at January 1, 2016

import statsmodels.api as sm
import os
# SARIMAX example
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pylab as plt
from sklearn.metrics import mean_squared_error
ticker='JHL.AX'
months=12
end = datetime.date.today()
day=end.day
year=end.year-months//12-1
month=months%12+1
start=datetime.datetime(year,month,day)
steps=3
disp=False
data=web.DataReader(ticker,"yahoo",start,end)['Adj Close']   

#y=data.resample('MS').mean()

#parameters=quickParameters(paramPath,ticker,y,steps=2,disp=False)
#parameters=selectParameters(ticker,y,steps=2,disp=True)
#['8CO.AX', 0, 0, 0, 0, 0, 0, 12, 't', -7.909122119923726, 0.03528654153968552]

#parameters=['MFG.AX', 0, 0, 1, 0, 1, 0, 12, 'n', 321.3071720270583, 0.486377988114156]
data[-1]
y=data.resample('MS').mean()

In [ ]:
train_y, test_y = y[:-steps], y[-steps:]
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [[x[0], x[1], x[2], 12] for x in list(itertools.product(p, d, q))]
# fit model
trend_c=['n','c','t','ct']
mses=pd.DataFrame()
error=pd.DataFrame()
parameters=[]
forcast=None
for param in pdq:
    for t in trend_c:
        for param_seasonal in seasonal_pdq:
            try:
                model = SARIMAX(train_y,
                                     order=param,
                                     seasonal_order=param_seasonal,
                                     trend=t,
                                     enforce_stationarity=False,
                                     enforce_invertibility=False)

                model_fit = model.fit(disp=False)

                pred=model_fit.forecast(steps=len(test_y))
    #            print(pred)
    #            print(test_y)
                #print('mse:{}'.format(mean_squared_error(test_y,pred)))
                mse=measure_rmse(test_y,pred)
                mses=mses.append([list(param)+list(param_seasonal)+[t]+[model_fit.aic]+[mse]],ignore_index=True)

                if parameters==[] or parameters[-1]>mse:
                    parameters=[ticker]+list(param)+list(param_seasonal)+[t]+[model_fit.aic]+[mse]
                    forcast=pred

            except :

                #print('ARIMA{} error:{}'.format(param,e))
                error=error.append([list(param)+list(param_seasonal)+[t]],ignore_index=True)
                pass
print(parameters)
#model_fit=model.fit(disp=0)
if disp:
    print('forcast::',forcast)
    pred_ci=pd.DataFrame(index=forcast.index)
    pred_ci['low'] = forcast-forcast*0.05
    pred_ci['upper'] = forcast+forcast*0.05


    #pred_ci.loc[y.index[-1]]=[y[-1],y[-1]]
    #pred_ci=pred_ci.sort_index()
    ax = y['2019-01-01':].plot(label='observed')
    forcast.plot(ax=ax, label='Forecast', alpha=.7)

    ax.fill_between(forcast.index,
                    pred_ci.iloc[:,0],
                    pred_ci.iloc[:,1], color='k', alpha=.1)

    ax.set_xlabel('Date')
    ax.set_ylabel(ticker)
    plt.legend()

    plt.show()


In [ ]:


#parameters=quickParameters(paramPath,ticker,y,steps=2,disp=False)
parameters=selectParameters(ticker,y,steps=4,disp=disp)



train_y, test_y = y[:-steps], y[-steps:]

p1,p2,t=parameters[1:4],parameters[4:8],parameters[8]
print(p1,p2,t)
model = SARIMAX(train_y,
     order=p1,
     seasonal_order=p2,
     trend=t,
     enforce_stationarity=False,
     enforce_invertibility=False)

model_fit = model.fit(disp=False)

forcast=model_fit.forecast(steps=len(test_y))
forcast=train_y[-1:].append(forcast)
pred_ci=pd.DataFrame(index=forcast.index)
pred_ci['low'] = forcast-forcast*0.05
pred_ci['upper'] = forcast+forcast*0.05


#pred_ci.loc[y.index[-1]]=[y[-1],y[-1]]
#pred_ci=pred_ci.sort_index()
ax = y['2019-01-01':].plot(label='observed')
forcast.plot(ax=ax, label='Forecast', alpha=.7)

ax.fill_between(forcast.index,
                pred_ci.iloc[:,0],
                pred_ci.iloc[:,1], color='k', alpha=.1)

ax.set_xlabel('Date')
ax.set_ylabel(ticker)
plt.legend()

plt.show()
print(train_y)
print(test_y)
print(forcast) 